In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

# Box-Constrained Quadratic Methods

Quadratic programming (QP) is the process of solving a special type of mathematical optimization problem—specifically, a (linearly constrained) quadratic optimization problem, that is, the problem of optimizing (minimizing or maximizing) a quadratic function of several variables subject to linear constraints on these variables. Quadratic programming is a particular type of nonlinear programming.

A quadratic function subject to linear constraints can be written as:

$$
\begin{align}
    \begin{aligned}
        \min_x \frac{1}{2}x^TQx + q^Tx \\
        \begin{split}
            \textrm {subject to } & lb \leq x \leq ub \\& Gx \leq h \\& A x = b
        \end{split}
    \end{aligned}
\end{align}
$$

For what about its usage in machine learning we are interested in a special case of this problem subject to equality and box-constraints:

$$
\begin{align}
    \begin{aligned}
        \min_x \frac{1}{2}x^TQx + q^Tx \\
        \begin{split}
            \textrm {subject to } & lb \leq x \leq ub \\& Ax = b
        \end{split}
    \end{aligned}
\end{align}
$$

## Projected Gradient

## Frank-Wolfe or Conditional Gradient

## Active Set

## Interior Point

## Lagrangian Dual Relaxation